In [ ]:
!git clone https://github.com/ggerganov/llama.cpp

In [ ]:
!cd llama.cpp && LLAMA_CUBLAS=1 make && pip install -r requirements/requirements-convert-hf-to-gguf.txt

In [ ]:
from huggingface_hub import snapshot_download

In [ ]:
model_name = "MODEL_HERE"

In [ ]:
methods = ['q4_k_m']

In [ ]:
base_model = "./original_model/"
quantized_path = "./quantized_model/"

In [ ]:
snapshot_download(repo_id=model_name, local_dir=base_model , local_dir_use_symlinks=False)
original_model = quantized_path+'/FP16.gguf'

In [ ]:
!mkdir ./quantized_model/

In [ ]:
!python llama.cpp/convert-hf-to-gguf.py ./original_model/ --outtype f16 --outfile ./quantized_model/FP16.gguf

In [ ]:
import os

In [ ]:
for m in methods:
    qtype = f"{quantized_path}/{m.upper()}.gguf"
    os.system("./llama.cpp/quantize "+quantized_path+"/FP16.gguf "+qtype+" "+m)

In [ ]:
! ./llama.cpp/main -m ./quantized_model/Q4_K_M.gguf -n 90 --repeat_penalty 1.0 --color -i -r "User:" -f llama.cpp/prompts/chat-with-bob.txt

In [ ]:
from huggingface_hub import HfApi, HfFolder, create_repo, upload_file

In [ ]:
model_path = "./quantized_model/Q4_K_M.gguf" # Your model's local path
repo_name = "REPONAME"  # Desired HF Hub repository name
repo_url = create_repo(repo_name, private=False)
hf_token =""

In [ ]:
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="Q4_K_M.gguf",
    repo_id="username/reponame",
    repo_type="model",
    token=hf_token
)